In [1]:
import os
import json
import uuid
import datetime
from sentence_transformers import SentenceTransformer
import faiss
import pickle
from ollama import Client

client = Client(host="http://localhost:11434")
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Load vector DB + metadata
index = faiss.read_index("vector.index")
with open("metadata.pkl", "rb") as f:
    metadata = pickle.load(f)

# Chat state
chat_history = []
MAX_TURNS = 10
session_id = str(uuid.uuid4())[:8]
chat_log_file = f"chat_log_{session_id}.json"


In [ ]:
def retrieve_context(query, k=5):
    vec = embedding_model.encode([query])
    distances, indices = index.search(vec, k)
    return [metadata[i] for i in indices[0]]

def build_prompt(query, context):
    context_text = "\n\n".join(
        f"Title: {doc['title']}\nURL: {doc['url']}\n\n{doc.get('cleaned_content', '')[:1000]}"
        for doc in context
    )
    return context_text


In [ ]:
def save_chat_log():
    if not chat_history:
        return
    with open(chat_log_file, "w", encoding="utf-8") as f:
        json.dump(chat_history, f, indent=2)

def reset_chat():
    global chat_history, session_id, chat_log_file
    chat_history = []
    session_id = str(uuid.uuid4())[:8]
    chat_log_file = f"chat_log_{session_id}.json"
    print(f"🔄 New session started: {session_id}")

def chat_with_mistral(query):
    global chat_history

    if query.strip().lower() == "clear":
        reset_chat()
        return "🧼 Chat memory cleared. New session started."

    now = datetime.datetime.now().isoformat()

    # Retrieve documents
    docs = retrieve_context(query)
    context = build_prompt(query, docs)

    # Memory history
    memory = "\n".join(
        f"[{ts}] User: {q}\n[{ts2}] Agent: {a}"
        for (q, a, ts, ts2) in chat_history[-MAX_TURNS:]
    )

    prompt = f"""
You are a helpful research assistant for DRDO analysts.

Previous conversation:
{memory}

Relevant context:
{context}

[{now}] User: {query}

Answer:"""

    response = client.chat(
        model="mistral",
        messages=[{"role": "user", "content": prompt}]
    )

    answer = response["message"]["content"]
    chat_history.append((query, answer, now, datetime.datetime.now().isoformat()))
    save_chat_log()
    return answer


In [ ]:
print(f"🤖 DRDO Agent [Session {session_id}] is ready. Type 'exit' or 'clear'.")

while True:
    user_input = input("\nYou: ")
    if user_input.lower() in ["exit", "quit"]:
        print("👋 Ending session.")
        break
    reply = chat_with_mistral(user_input)
    print("\nAgent:", reply)


🤖 DRDO Agent [Session a615718f] is ready. Type 'exit' or 'clear'.
